In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df=pd.read_csv(r"C:\Users\User\Desktop\end-to-end_ml_project\FPL_best_xi\data\raw\all_seasons_fpl.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_12056\1358467820.py:1: DtypeWarning: Columns (34,35,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\User\Desktop\end-to-end_ml_project\FPL_best_xi\data\raw\all_seasons_fpl.csv")


In [3]:
df.shape

(162028, 54)

In [4]:
print(df.head())

                    name  assists  bonus  bps  clean_sheets  creativity  \
0    Aaron_Cresswell_376        0      0    7             0         1.5   
1       Aaron_Lennon_430        0      0    3             0         0.0   
2         Aaron_Mooy_516        0      0    0             0         0.0   
3     Aaron_Ramsdale_494        0      0   11             0         0.0   
4  Aaron_Wan-Bissaka_122        0      2   34             1        16.1   

   element  fixture  goals_conceded  goals_scored  ...  mng_draw  \
0      376        8               5             0  ...       NaN   
1      430        3               0             0  ...       NaN   
2      516        7               0             0  ...       NaN   
3      494        2               1             0  ...       NaN   
4      122        9               0             0  ...       NaN   

   mng_goals_scored mng_loss  mng_underdog_draw  mng_underdog_win  mng_win  \
0               NaN      NaN                NaN               

In [5]:
keep_cols = [
    'name', 'position', 'team', 'season', 'gameweek',
    'total_points', 'minutes', 'goals_scored', 'assists',
    'clean_sheets', 'goals_conceded', 'saves', 'yellow_cards',
    'red_cards', 'bonus', 'bps', 'value', 'selected', 'transfers_in',
    'transfers_out'
]
available_cols = [c for c in keep_cols if c in df.columns]
df = df[available_cols].copy()

In [6]:
num_cols = [c for c in df.columns if c not in ['name','team','season','position']]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

In [7]:
df = df[df['minutes'] > 0].reset_index(drop=True)

In [8]:
df.shape

(68240, 20)

In [9]:
pos_map = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}
if df['position'].dtype != 'object':
    df['position'] = df['position'].map(pos_map)

In [10]:
df['team_code'] = df['team'].astype('category').cat.codes
df['pos_code'] = df['position'].astype('category').cat.codes
df['season_code'] = df['season'].astype('category').cat.codes

In [11]:
rolling_features = ['total_points', 'minutes', 'goals_scored', 'assists', 'clean_sheets']
window_sizes = [3, 5, 10]

df = df.sort_values(['name', 'season', 'gameweek'])

for feat in rolling_features:
    for w in window_sizes:
        df[f'{feat}_roll{w}'] = (
            df.groupby('name')[feat]
              .transform(lambda x: x.shift(1).rolling(window=w, min_periods=1).mean())
        )

# Example lag feature (last GW points)
df['last_points'] = df.groupby('name')['total_points'].shift(1).fillna(0)

In [12]:
df['value'] = df['value'] / 10  # Convert to actual value in millions

In [13]:
df['form_points'] = df['total_points_roll3'] +0.3*df['total_points_roll5'] +0.2*df['total_points_roll10']

In [14]:
output_path = "C:\\Users\\User\\Desktop\\end-to-end_ml_project\\FPL_best_xi\\data\\processed\\player_features.csv"
df.to_csv(output_path, index=False)

In [15]:
print(" Saved cleaned + feature dataset →", output_path)
print(" Final shape:", df.shape)
print(" Example columns:", df.columns[:15].tolist())

 Saved cleaned + feature dataset → C:\Users\User\Desktop\end-to-end_ml_project\FPL_best_xi\data\processed\player_features.csv
 Final shape: (68240, 40)
 Example columns: ['name', 'position', 'team', 'season', 'gameweek', 'total_points', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves', 'yellow_cards', 'red_cards', 'bonus']
